<br><br><br><br><br>

# Day 1 Homework

<br><br><br><br><br>

<br><br><br><br><br>

**You're going to implement a machine learning algorithm in Numpy.**

<br>

_(Don't worry—it's not that hard!)_

<br><br><br><br><br>

In [ ]:
%matplotlib inline
import numpy, matplotlib.pyplot                                 # Setting up the problem...
truth = numpy.random.normal(0, 3, (5, 2))                       # 5 random, 2D cluster centers
data = []
for x, y in truth:
    data.append(numpy.empty((numpy.random.poisson(100), 2)))    # about 100 points in each
    data[-1][:, 0] = numpy.random.normal(x, truth.std(axis=0)[0]/3, len(data[-1]))
    data[-1][:, 1] = numpy.random.normal(y, truth.std(axis=0)[1]/3, len(data[-1]))
for cluster in data:
    matplotlib.pyplot.scatter(cluster[:, 0], cluster[:, 1]);    # visualize them separately

In [ ]:
# The observed data are not labeled by cluster: the task is to identify centers of unlabeled data.

observed = numpy.concatenate(data)
numpy.random.shuffle(observed)

matplotlib.pyplot.scatter(observed[:, 0], observed[:, 1]);





In [ ]:
# The K-means algorithm is well suited to this task: run it from Scikit-Learn.

import sklearn.cluster

kmeans = sklearn.cluster.KMeans(n_clusters=5).fit(observed)

matplotlib.pyplot.scatter(observed[:, 0], observed[:, 1])
matplotlib.pyplot.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                          s=100, facecolors="none", edgecolors="darkorange", linewidths=3);


In [ ]:
# Implementing K-means ourselves...
# Step 1: make an initial guess by randomly selecting 5 points from the dataset.
def guess(observed):
    indexes = numpy.random.choice(len(observed), 5, replace=False)
    return numpy.stack([observed[i] for i in indexes])

centers = guess(observed)
matplotlib.pyplot.scatter(observed[:, 0], observed[:, 1])
matplotlib.pyplot.scatter(centers[:, 0], centers[:, 1],
                          s=100, facecolors="none", edgecolors="darkorange", linewidths=3);

In [ ]:
# Step 2: associate each data point with the cluster center that it is closest to.
def expectation(centers, observed):
    clusters = [[] for i in range(5)]
    for x, y in observed:
        distances = [(x - center[0])**2 + (y - center[1])**2 for center in centers]
        for i in range(5):
            if all(distances[i] <= distances[j] for j in range(5)): clusters[i].append((x, y))
    return [numpy.array(cluster) for cluster in clusters]

for cluster in expectation(centers, observed): matplotlib.pyplot.scatter(cluster[:, 0], cluster[:, 1]);

In [ ]:
# Step 3: update each cluster center to the average position of the data associated with it.
def minimization(clusters):
    xcenters = [sum(cluster[:, 0])/len(cluster) for cluster in clusters]
    ycenters = [sum(cluster[:, 1])/len(cluster) for cluster in clusters]
    return numpy.array(list(zip(xcenters, ycenters)))

centers = minimization(expectation(centers, observed))
matplotlib.pyplot.scatter(observed[:, 0], observed[:, 1])
matplotlib.pyplot.scatter(centers[:, 0], centers[:, 1],
                          s=100, facecolors="none", edgecolors="darkorange", linewidths=3);

### Homework:

Replace the implementations of

   * `guess(observed)`
   * `expectation(centers, observed)`
   * `minimization(clusters)`

with Numpythonic alternatives. Although <tt>guess</tt> can only be improved in style, the other two have execution speed issues: they run Python for loops over the entire dataset (`for x, y in observed` in <tt>expectation</tt> and implicitly using `sum(...)` in <tt>minimization</tt>).

In your improved implementation, you're allowed to run Python for loops over the number of clusters (`5`) but not the number of observed data points (`~500`). Be aware that list/generator comprehensions and constructors that take iterators (e.g. `all`, `any`, `sum`, `list`, `zip`) count as Python for loops.

The <tt>guess</tt> implementation can be reduced to 1 line, <tt>expectation</tt> to two lines, and <tt>minimization</tt> to 1 line (without semicolons, of course). I don't know of any way to do <tt>expectation</tt> in one line without repeating calculations.